## Read short story for tokenzation

### Step 1: Create tokens

In [8]:
with open("the-verdict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

# Print total number of characters
print("Total number of characters:", len(raw_text))

# print first 100 characters
print(raw_text[:99])


Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [9]:
# Use regular expression library
import re

text = "Hello world! This is a test, working now."

# Split based on white spaces: use \s as regex
result = re.split(r'\s', text) # This will remove the white space

print(result)

result = re.split(r'(\s)', text) # This will include the white spaces
print(result)


['Hello', 'world!', 'This', 'is', 'a', 'test,', 'working', 'now.']
['Hello', ' ', 'world!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test,', ' ', 'working', ' ', 'now.']


In [10]:
# Now we want to include punctuation characters (commas full stops and exclamation marks) also as separate strings
result = re.split(r'(\W)', text)
print(result)

['Hello', ' ', 'world', '!', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', ',', '', ' ', 'working', ' ', 'now', '.', '']


In [11]:
# Remove empty strings and white space characters now
result = [item for item in result if item.strip()]

print(result)

['Hello', 'world', '!', 'This', 'is', 'a', 'test', ',', 'working', 'now', '.']


In [12]:
# for the book text, there are ? -- : _ ; ' " () are also present. Hence we need to consider them while splitting
# Hence our tokenizer code is as follows, as of now. This is a simple tokenizer. 
# For LLMs, different tokenizer scheme is used which we will see later
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [t for t in result if t.strip()]
print (len(preprocessed))

11


### Step 2: Creating token IDs

In [13]:
# Get all unique words and sorted
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

11


In [14]:
# Create a vocabulary
vocab = {}
counter = 0
for word in all_words:
    vocab[word] = counter
    counter = counter + 1

items = list(vocab.items())[:20]
for key,value in items:
    print(f"({key}, {value})")

(!, 0)
(,, 1)
(., 2)
(Hello, 3)
(This, 4)
(a, 5)
(is, 6)
(now, 7)
(test, 8)
(working, 9)
(world, 10)


In [ ]:
# Now we build a simple tokenizer class that will be used to encode the given text to feed IDs to LLM and decode the Token IDs from LLM to convert back to text.
class SimpleTokenizerV1:
    def __init__(self, vocab):
        pass

    def encode(self, text):
        pass

    def decode(self, ids):
        pass